# Preliminary steps

Transform the csv files into a zipped npy format with a reduced size 

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

In [ ]:
import os.path as op
import os

### Time-steps and files

In [ ]:
home_dir = op.expanduser('~')
base_dir = op.join(home_dir, 'simdb', 'data', 'asinning')
data_dir_name = '3D-Aramis'
time_F_dic_file_name = 'S16-1_Kraft.csv'
data_path = op.join(base_dir, 'S16')
csv_data_dir = op.join(data_path, data_dir_name)
time_F_dic_file = op.join(data_path, time_F_dic_file_name)
npz_file = op.join(data_path, 'S16.npz')

In [ ]:
csv_data_dir

In [ ]:
time_F_dic_file

## Read the load deflection curve

In [ ]:
time_F_w_m = np.array(
    pd.read_csv(time_F_dic_file, decimal=",", skiprows=1,
    delimiter=';'), dtype=np.float_
)

In [ ]:
pxyze_file_T = [op.join(csv_data_dir, dic_file) for dic_file in os.listdir(csv_data_dir)]

## Read from csv files

In [ ]:
pxyze_file_T[-1]

In [ ]:
pxyze_list = [
    np.loadtxt(pxyz_file, dtype=np.float_,
               skiprows=6, delimiter=';', usecols=(0, 1, 2, 3, 4))
    for pxyz_file in pxyze_file_T
]
n_T = len(pxyze_list)
n_T -= 10

## Do all the data records for each time step have the same length?

Since some points were lost during the cracking, not all records are the same.
The next code identifies all points that exist in all time steps and ignores those points
that were lost during the cracking.

In [ ]:
# Get all point ids in all time steps
P_list = [np.array(pxyz[:, 0], dtype=np.int_)
          for pxyz in pxyze_list[:n_T]]
# Maximum number of points ocurring in one of the time steps to allocate the space
max_n_P = np.max(np.array([np.max(P_) for P_ in P_list])) + 1
# Only points occurring in all time steps are considered
P_Q = P_list[0]
for P_next in P_list[1:]:
    P_Q = np.intersect1d(P_Q, P_next)
# Define the initial configuration
X_TPa = np.zeros((n_T, max_n_P, 3), dtype=np.float_)
for T in range(n_T):
    X_TPa[T, P_list[T]] = pxyze_list[T][:, 1:4]
X_TQa = X_TPa[:, P_Q, :]

In [ ]:
X_0Qa = X_TQa[0]
U_TQa =  X_TQa - X_0Qa[None, ...]

## Compressed array data files `npz` and `npy`

Write the compressed numpy file containing several arrays

In [ ]:
np.savez('S16.npz', X_0Qa=X_0Qa, U_TQa=U_TQa)